# YOLOv8 Segmentation

In [ ]:
pip install ultralytics
from ultralytics import YOLO
from matplotlib import pyplot as plt
import cv2
import os
import numpy as np
from pathlib import Path

model = YOLO('yolov8x-seg.yaml')
model = YOLO('yolov8x-seg.pt')

from google.colab import drive
drive.mount('/content/drive')

%cat /content/drive/MyDrive/yolo_fix/yolo_dataset_noaug/data.yaml

import yaml
with open("/content/drive/MyDrive/yolo_fix/yolo_dataset/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

project = "/content/drive/MyDrive/yolo_fix/results"
name = "xl250"

results = model.train(data='/content/drive/MyDrive/yolo_fix/yolo_dataset/data.yaml',
                      project=project,
                      name=name,
                      epochs=250)

best_model = YOLO('/content/drive/MyDrive/yolo_fix/results/xl250/weights/best.pt')

#Evaluate
metrics = best_model.val(data='/content/drive/MyDrive/yolo_fix/testing/0/data.yaml',split='test')
print(metrics.box.map)

#Removing background
output_dir = '/content/drive/MyDrive/classifier_dataset'
Path(output_dir).mkdir(parents=True, exist_ok=True)

def process_image(image_path, model, output_dir):
    image = cv2.imread(image_path)

    resized_image = cv2.resize(image, (800, 800))

    results = model(resized_image, conf=0.2, iou=0.3)

    masks = np.sum(results[0].masks.data.cpu().numpy(), axis=0)

    masks_resized = cv2.resize(masks.astype(np.uint8), (resized_image.shape[1], resized_image.shape[0]))

    masks_binary = (masks_resized > 0.5).astype(np.uint8)

    processed_image = resized_image.copy()
    processed_image[masks_binary == 0] = 255

    filename = os.path.splitext(os.path.basename(image_path))[0]
    cv2.imwrite(os.path.join(output_dir, f"{filename}.jpg"), processed_image)

dataset_dir = '/content/drive/MyDrive/Dataset/Hanya Daun/Vertikal'
image_files = [f for f in os.listdir(dataset_dir)]

for image_file in image_files:
    image_path = os.path.join(dataset_dir, image_file)
    process_image(image_path, best_model, output_dir)

# EfficientNet

In [ ]:
pip install keras_tuner
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop
import keras_tuner as kt

base_dir = '/content/drive/MyDrive/classifier_dataset_splitted'
train_path = os.path.join(base_dir, 'train')
valid_path = os.path.join(base_dir, 'validation')

img_height, img_width = 456, 456
num_classes = 5

# Data Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=8,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_path,
    target_size=(img_height, img_width),
    batch_size=8,
    class_mode='categorical'
)

#Hyperparameter tuning
def build_model(hp):
    base_model = EfficientNetB5(weights='imagenet', include_top=False)
    to_freeze = hp.choice('freeze', [42,116,190,294,397,531,574,577])
    for layer in base_model.layers[:to_freeze]:
      layer.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(hp.Choice('dropout 1', [0.2, 0.5]))(x)
    x = Dense(hp.Choice('dense', [512, 1024]), activation='relu')(x)
    x = Dropout(hp.Choice('dropout 2', [0.2, 0.5]))(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(5, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    optimizer_choice = hp.Choice('optimizer', ['adam', 'rmsprop'])

    if optimizer_choice == 'adam':
      optimizer = tf.keras.optimizers.Adam(learning_rate = hp.Choice('learning_rate', [1e-3, 1e-4, 5e-5, 1e-5]))
    else:
      optimizer = tf.keras.optimizers.RMSprop(learning_rate = hp.Choice('learning_rate', [1e-3, 1e-4, 5e-5, 1e-5]))

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

tuner = kt.GridSearch(
    build_model,
    objective='val_loss',
    max_trials = 512,
    overwrite=True,
    directory='hyperparams_tuning',
    project_name='hyperparams_tuning')

tuner.search(
    train_generator, epochs=20,
    validation_data=valid_generator)

tuner.results_summary()

#Train model with best hps
best_hps = tuner.get_best_hyperpamrameters(5)
model = build_model(best_hps[0])

history = model.fit(train_generator, epochs=50, validation_data=test_generator)

#Plot training and validation accuracy
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()